# Prepreocessing

## Import and strip data

In [3]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

In [4]:
#import data
student = pd.read_csv('hsls_16_student_v1_0.csv')

In [6]:
# Define target columns
targets = ['X3TGPAENG',
               'X3TGPAMAT',
               'X3TGPASCI',
               'X3TGPAHISCI',
               'X3TGPASOCST',
               'X3TGPAART',
               'X3TGPALANG',
               'X3TGPAACAD',
               'X3TAFGPATOT',
               'X4HSCOMPSTAT',
               'X4EVRAPPCLG',
               'X4PS1SELECT',
               'X4PS1DEGTYPE1',
               'X4PSLFSTFB16'
               ]

In [9]:
# Define entries which indicate a missing or skipped question
impute_values = [np.nan, -4, -5, -6, -7, -8, -9]

In [8]:
array = []

# Collect columns that contain redacted information
for x in student.columns:
    if student[x][0] == -5:
        array.append(x)

student_no_null = student.drop(columns=array)

array = []

# Collect columns that contain weights used by NESC (Not usable in analysis)
for x in student_no_null.columns:
    if x[0] == "W":
        array.append(x)


student_no_null_or_weight = student_no_null.drop(columns=array)
X_and_y = pd.read_csv('x and y Columns.csv')
X= student_no_null_or_weight[X_and_y['X'].tolist()]

y = student_no_null_or_weight[X_and_y['y'].dropna().tolist()]
for column in targets:
    X.loc[:, column] = y[column]

In [15]:
# Define function to replace missing values with NaN
def replace_missing_values(df):
    df.replace(impute_values, value=np.nan, inplace=True)
 
# Define function to impute on any missing values in column   
def run_all_imputers(column):
    for val in impute_values:
        imputer = SimpleImputer(missing_values=val, strategy='median')
        column = imputer.fit_transform(column)
    return column


# Define function to verify that only valid values are present in column
def check_column(column):
    for val in impute_values:
        if val in column:
            print('Found {0} in column!!'.format(val))

In [13]:
# Define function to perform imputation or hot encoding on a column
def impute_column(df, feature, threshold=10, enable_scaling=False):
    column = df[[feature]]
    unique_array = df[feature].unique()
    valid_uniques = [x for x in unique_array if x >= 0]
    if len(unique_array) > threshold or len(valid_uniques) <= 2:
        print('Imputing {0}'.format(feature))
        imputed_column = run_all_imputers(column)
        check_column(imputed_column)
        if enable_scaling:
            scalar = StandardScaler()
            imputed_column = scalar.fit_transform(imputed_column)
        df.loc[:, feature] = imputed_column
        return df
    else:  # encode
        cat = [[x for x in unique_array if x >= 0]]
        cat = [sorted(cat[0])]
        if len(cat[0]) <= 2:
            return df
        print("Encoding {0}".format(feature))
        new_df = pd.get_dummies(df, prefix=feature, columns=[feature], drop_first=False)
        df.drop(feature, axis=1, inplace=True)
        for column in new_df.columns:
            df.loc[:, column] = new_df[column]

## Run imputation and hot encoding

In [16]:
replace_missing_values(X)
print(X.shape)
columns = list(X)
for column in columns:
    if column in targets:
        continue
    X_imputed = impute_column(X, column, 15, enable_scaling=True)
X.to_csv('preprocessed_data.csv')

(23503, 588)
Imputing X1SEX
Encoding X1RACE


/Users/robmisasi/Documents/University/Classes/EE_461P/TermProject/env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/robmisasi/Documents/University/Classes/EE_461P/TermProject/env/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/Users/robmisasi/Documents/University/Classes/EE_461P/TermProject/env/lib/python3.6/site-packages/pandas/core/indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try us

Imputing X1HISPANIC
Imputing X1WHITE
Imputing X1BLACK
Encoding X1DUALLANG


/Users/robmisasi/Documents/University/Classes/EE_461P/TermProject/env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/robmisasi/Documents/University/Classes/EE_461P/TermProject/env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Users/robmisasi/Documents/University/Classes/EE_461P/TermProject/env/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is

KeyboardInterrupt: 